[2023/06/14 03:23:49] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\a23a2/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='slow', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\a23a2/.paddleocr/whl\\rec\\en\\en_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, re

In [1]:
import os
import time
import random
import math
import sys
from glob import glob
from typing import List, Tuple
from difflib import SequenceMatcher
import ipywidgets as widgets
from IPython.display import display
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import imageio

from tqdm import tqdm, trange

from sklearn import cluster
from sklearn.mixture import GaussianMixture as GMM
from sklearn.cluster import KMeans
from sklearn import mixture
from sklearn import manifold
from typing import List

from paddleocr import PaddleOCR, draw_ocr
from paddleocr import paddleocr
import speech_recognition as sr
import re

import time
import cv2
import contact_to_server
import serial
import numpy as np
r = sr.Recognizer()
YAPS=0
pattern = r"[A-Z]\d{8}"

0

In [3]:
#idchack
def ocr_info_to_location(ocr_info, acc=0):
    bboxs = None
    for info in ocr_info:
        if info[1][1] >= acc:
            info[0] = np.array(info[0], dtype=np.uint32)
            bbox = np.array([info[0][:, 1].min(), info[0][:, 0].min(
            ), info[0][:, 1].max(), info[0][:, 0].max(), info[1][0]])
            bboxs = np.array(
                [bbox]) if bboxs is None else np.vstack(
                (bboxs, bbox))
    #['174' '143' '229' '260' '姓老张学彬']
    return bboxs
ocr = PaddleOCR(use_angle_cls=True,
                lang='en',
                use_gpu=False,
                enable_mkldnn=True,
                # rec_batch_num=30,
                det_max_side_len=3200,
                drop_score=0,  
                det_db_score_mode='slow',
                # det_db_box_thresh=0,
                # det_db_thresh=0.01
               )

try:
    ser = serial.Serial('COM8', 9600)  # 根据实际情况修改串口号
except:
    print("Have ser")

# 创建VideoCapture对象，参数为摄像头索引号（通常为0，表示默认摄像头）

ChackID=0
while True:
    # 从摄像头捕获一帧图像
    with sr.Microphone() as source:
        print("請開始說話...")
        audio = r.listen(source)
    try:
        text = r.recognize_google(audio, language='zh-TW')
    except: 
        print("again")
        continue
    print("辨識結果：" + text)
    if(text=="驗證"):
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)  # 设置帧宽度
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)  # 设置帧高度
        while(True):
            
            ret, frame = cap.read()
            H=frame.shape[0]
            W=frame.shape[1]
            CH=H//2
            CW=W//2
            center=(CW,CH)
            OG=frame.copy()
            # frame=cv2.flip(frame, 1)
            # 如果成功捕获图像，则处理并显示
            cv2.rectangle(frame, (CW-400,CH-250), (CW+400,CH+250), (255, 0, 0), 5)
            # cv2.circle(frame,(50, 50), 15, (255, 0, 0), -1) # blue
            cv2.imshow('Camera', frame)
            if (ser.in_waiting):  # 如果串口接收缓冲区有数据
                data = ser.readline().decode().rstrip()  # 读取一行数据并移除换行符
                img=OG[CH-250:CH+250,CW-400:CW+400]
                print("ABC")
                break
            # 按下ESC键退出循环
            if cv2.waitKey(1) == 27:
                break
        cap.release()
        cv2.destroyAllWindows()
        BOX=ocr_info_to_location(ocr.ocr(img)[0])
    elif (text=="拍攝"):
# 释放VideoCapture对象和窗口
        try:
            for i in BOX:
                match = re.match(pattern, i[4])
                if match:
                    string=i[4]
                    # print("符合正則表達式")
            # ID=contact_to_server.get_id_list()
            ID=["M11107509"]
            for j in ID:
                if j==string:
                    YAPS=1
                    print("OK")
                
            if(YAPS==1):
                cap = cv2.VideoCapture(0)
                cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)  # 设置帧宽度
                cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)  # 设置帧高度
                ChackID=0
                frames = []
                Catch=0
                start_time = None
                while True:
                    # 从摄像头捕获一帧图像
                    ret, frame = cap.read()
                    H=frame.shape[0]
                    W=frame.shape[1]
                    CH=H//2
                    CW=W//2
                    center=(CW,CH)
                    OG=frame.copy()
                    # frame=cv2.flip(frame, 1)
                    # 如果成功捕获图像，则处理并显示
                    cv2.rectangle(frame, (CW-350,CH-350), (CW+350,CH+350), (255, 0, 0), 5)
                    # cv2.circle(frame,(50, 50), 15, (255, 0, 0), -1) # blue
                    cv2.imshow('Camera', frame)
                    if (ser.in_waiting and Catch==0):  # 如果串口接收缓冲区有数据
                        data = ser.readline().decode().rstrip()  # 读取一行数据并移除换行符
                        frames = []
                        print("START")
                        start_time =time.time()
                        Catch=1
                    if(Catch==1):
                        frames.append(OG[CH-350:CH+350,CW-350:CW+350])
                    
                    # 按下ESC键退出循环
                    if cv2.waitKey(1) == 27:
                        break
                    if start_time is not None and time.time() - start_time >= 5 and Catch==1:
                        Catch=0
                        print("STOP")
                        break

                # 释放VideoCapture对象和窗口
                cap.release()
                cv2.destroyAllWindows()
                frames=np.array(frames)
                imgs = np.stack(frames[[0,75,151]], axis=3)
                contact_to_server.update_person_img(imgs)
                contact_to_server.decrease_waiting_person()
            else:
                print("error")
        except:
            print("Error")
                






[2023/06/14 10:51:31] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\a23a2/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='slow', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\a23a2/.paddleocr/whl\\rec\\en\\en_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, re

KeyboardInterrupt: 

In [7]:
np.array(frames).shape

(151, 700, 700, 3)

In [12]:
imgs.shape

(700, 700, 3, 3)

In [8]:
ser.close()

In [1]:
#idchack
import speech_recognition as sr
import time
import cv2
import contact_to_server
import serial
import numpy as np
from paddleocr import PaddleOCR, draw_ocr
from paddleocr import paddleocr
YAPS=0
r = sr.Recognizer()
def ocr_info_to_location(ocr_info, acc=0):
    bboxs = None
    for info in ocr_info:
        if info[1][1] >= acc:
            info[0] = np.array(info[0], dtype=np.uint32)
            bbox = np.array([info[0][:, 1].min(), info[0][:, 0].min(
            ), info[0][:, 1].max(), info[0][:, 0].max(), info[1][0]])
            bboxs = np.array(
                [bbox]) if bboxs is None else np.vstack(
                (bboxs, bbox))
    #['174' '143' '229' '260' '姓老张学彬']
    return bboxs
ocr = PaddleOCR(use_angle_cls=True,
                lang='en',
                use_gpu=False,
                enable_mkldnn=True,
                # rec_batch_num=30,
                det_max_side_len=3200,
                drop_score=0,  
                det_db_score_mode='slow',
                # det_db_box_thresh=0,
                # det_db_thresh=0.01
               )

try:
    ser = serial.Serial('COM8', 9600)  # 根据实际情况修改串口号
except:
    print("Have ser")

# 创建VideoCapture对象，参数为摄像头索引号（通常为0，表示默认摄像头）
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)  # 设置帧宽度
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)  # 设置帧高度
ChackID=0
while True:
    # 从摄像头捕获一帧图像
    text=""
    with sr.Microphone() as source:
        print("請開始說話...")
        audio = r.listen(source)
    text = r.recognize_google(audio, language='zh-TW')
    print("辨識結果：" + text)
    if(text=="驗證"):
        while(True):
            ret, frame = cap.read()
            H=frame.shape[0]
            W=frame.shape[1]
            CH=H//2
            CW=W//2
            center=(CW,CH)
            OG=frame.copy()
            # frame=cv2.flip(frame, 1)
            # 如果成功捕获图像，则处理并显示
            cv2.rectangle(frame, (CW-400,CH-250), (CW+400,CH+250), (255, 0, 0), 5)
            # cv2.circle(frame,(50, 50), 15, (255, 0, 0), -1) # blue
            cv2.imshow('Camera', frame)
            if (ser.in_waiting):  # 如果串口接收缓冲区有数据
                data = ser.readline().decode().rstrip()  # 读取一行数据并移除换行符
                img=OG[CH-250:CH+250,CW-400:CW+400]
                print("ABC")
                break
            # 按下ESC键退出循环
            if cv2.waitKey(1) == 27:
                break

# 释放VideoCapture对象和窗口
cap.release()
cv2.destroyAllWindows()
BOX=ocr_info_to_location(ocr.ocr(img)[0])
import re
pattern = r"[A-Z]\d{8}"

for i in BOX:
    match = re.match(pattern, i[4])
    if match:
        string=i[4]
        print("符合正則表達式")
    else:
        print("不符合正則表達式")
ID=contact_to_server.get_id_list()
for i in ID:
    if i==string:
        YAPS=1
        print("OK")
if(YAPS==1):
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)  # 设置帧宽度
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)  # 设置帧高度
    ChackID=0
    frames = []
    Catch=0
    start_time = None
    while True:
        # 从摄像头捕获一帧图像
        if (ser.in_waiting and ChackID==0):  # 如果串口接收缓冲区有数据
            data = ser.readline().decode().rstrip()  # 读取一行数据并移除换行符
            print("Sure")  # 输出数据
            ChackID=1
        if(ChackID==1):
            ret, frame = cap.read()
            H=frame.shape[0]
            W=frame.shape[1]
            CH=H//2
            CW=W//2
            center=(CW,CH)
            OG=frame.copy()
            # frame=cv2.flip(frame, 1)
            # 如果成功捕获图像，则处理并显示
            cv2.rectangle(frame, (CW-350,CH-350), (CW+350,CH+350), (255, 0, 0), 5)
            # cv2.circle(frame,(50, 50), 15, (255, 0, 0), -1) # blue
            cv2.imshow('Camera', frame)
            if (ser.in_waiting and Catch==0):  # 如果串口接收缓冲区有数据
                data = ser.readline().decode().rstrip()  # 读取一行数据并移除换行符
                frames = []
                print("START")
                start_time =time.time()
                Catch=1
            if(Catch==1):
                frames.append(OG[CH-350:CH+350,CW-350:CW+350])
            
            # 按下ESC键退出循环
            if cv2.waitKey(1) == 27:
                break
            if start_time is not None and time.time() - start_time >= 5 and Catch==1:
                Catch=0
                print("STOP")
                break

    # 释放VideoCapture对象和窗口
    cap.release()
    cv2.destroyAllWindows()
    frames=np.array(frames)
    imgs = np.stack(frames[[0,75,151]], axis=3)
    contact_to_server.update_person_img(imgs)
else:
    print("Error")
    

[2023/06/14 03:31:39] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\a23a2/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='slow', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\a23a2/.paddleocr/whl\\rec\\en\\en_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, re

: 

: 

In [89]:
frames=np.array(frames)
imgs = np.stack(frames[[0,75,151]], axis=3)

In [93]:
contact_to_server.update_person_img(imgs)

In [95]:
contact_to_server.get_car_information()

[]

In [74]:
ironman_arange = np.arange(0,151,75) #建立一個依序填滿的陣列 0到20間隔為2

In [73]:
imgs.shape

(700, 700, 3, 152)

In [65]:
ID

[]

In [25]:
#錄製段落
import cv2
import serial
import time 
# ser = serial.Serial('COM8', 9600)  # 根据实际情况修改串口号

# 创建VideoCapture对象，参数为摄像头索引号（通常为0，表示默认摄像头）
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)  # 设置帧宽度
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)  # 设置帧高度
ChackID=0
frames = []
Catch=0
start_time = None
while True:
    # 从摄像头捕获一帧图像
    if (ser.in_waiting and ChackID==0):  # 如果串口接收缓冲区有数据
        data = ser.readline().decode().rstrip()  # 读取一行数据并移除换行符
        print("Sure")  # 输出数据
        ChackID=1
    if(ChackID==1):
        ret, frame = cap.read()
        H=frame.shape[0]
        W=frame.shape[1]
        CH=H//2
        CW=W//2
        center=(CW,CH)
        OG=frame.copy()
        # frame=cv2.flip(frame, 1)
        # 如果成功捕获图像，则处理并显示
        cv2.rectangle(frame, (CW-350,CH-350), (CW+350,CH+350), (255, 0, 0), 5)
        # cv2.circle(frame,(50, 50), 15, (255, 0, 0), -1) # blue
        cv2.imshow('Camera', frame)
        if (ser.in_waiting and Catch==0):  # 如果串口接收缓冲区有数据
            data = ser.readline().decode().rstrip()  # 读取一行数据并移除换行符
            frames = []
            print("START")
            start_time =time.time()
            Catch=1
        if(Catch==1):
            frames.append(OG[CH-350:CH+350,CW-350:CW+350])
        
        # 按下ESC键退出循环
        if cv2.waitKey(1) == 27:
            break
        if start_time is not None and time.time() - start_time >= 5 and Catch==1:
            Catch=0
            print("STOP")

# 释放VideoCapture对象和窗口
cap.release()
cv2.destroyAllWindows()

Sure
START
STOP


KeyboardInterrupt: 

In [38]:
import contact_to_server
ID=contact_to_server.get_id_list()

In [44]:
ID
print(string)
print(ID[3])

M11107509
M11107509


In [45]:
print(ID[3]==string)

True


In [51]:
frames.shape

AttributeError: 'list' object has no attribute 'shape'

In [62]:
contact_to_server.update_person_img(imgs)

In [60]:
X=contact_to_server.get_person_img()

In [61]:
X

[]

In [46]:
for i in ID:
    if i==string:
        print("OK")

OK


In [54]:
imgs = np.stack(frames[0:2], axis=3)

In [58]:
imgs.shape

(700, 700, 3, 2)

In [53]:
imgs = np.stack(frames, axis=3)

AxisError: axis 3 is out of bounds for array of dimension 3

In [29]:
imgs.shape

(700, 700, 3, 151)

In [13]:
import serial

# 打开串口
# ser = serial.Serial('COM8', 9600)  # 根据实际情况修改串口号

while True:
    if ser.in_waiting:  # 如果串口接收缓冲区有数据
        data = ser.readline().decode().rstrip()  # 读取一行数据并移除换行符
        print("sure")  # 输出数据

sure
sure
sure


KeyboardInterrupt: 

In [ ]:
import cv2

# 创建VideoCapture对象，参数为摄像头索引号（通常为0，表示默认摄像头）
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)  # 设置帧宽度
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)  # 设置帧高度
while True:
    # 从摄像头捕获一帧图像
    ret, frame = cap.read()

    # 如果成功捕获图像，则处理并显示
    if ret:
        # 在图像上进行其他处理（例如，灰度化、边缘检测等）
        # ...

        # 显示图像
        cv2.imshow('Camera', frame)

    # 按下ESC键退出循环
    if cv2.waitKey(1) == 27:
        break

# 释放VideoCapture对象和窗口
cap.release()
cv2.destroyAllWindows()

In [ ]:
frame.shape

In [ ]:
import serial

# 打开串口
ser = serial.Serial('COM8', 9600)  # 根据实际情况修改串口号

while True:

    if ser.in_waiting:  # 如果串口接收缓冲区有数据
        data = ser.readline().decode().rstrip()  # 读取一行数据并移除换行符
        print("Sure")  # 输出数据

In [1]:
import cv2
import serial
ser = serial.Serial('COM8', 9600)  # 根据实际情况修改串口号

# 创建VideoCapture对象，参数为摄像头索引号（通常为0，表示默认摄像头）
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)  # 设置帧宽度
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)  # 设置帧高度
ChackID=0
while True:
    # 从摄像头捕获一帧图像
    if (ser.in_waiting and ChackID==0):  # 如果串口接收缓冲区有数据
        data = ser.readline().decode().rstrip()  # 读取一行数据并移除换行符
        print("Sure")  # 输出数据
        ChackID=1
    if(ChackID==1):
        ret, frame = cap.read()
        H=frame.shape[0]
        W=frame.shape[1]
        CH=H//2
        CW=W//2
        center=(CW,CH)
        OG=frame.copy()
        # frame=cv2.flip(frame, 1)
        # 如果成功捕获图像，则处理并显示
        cv2.rectangle(frame, (CW-400,CH-250), (CW+400,CH+250), (255, 0, 0), 5)
        # cv2.circle(frame,(50, 50), 15, (255, 0, 0), -1) # blue
        cv2.imshow('Camera', frame)
        if (ser.in_waiting):  # 如果串口接收缓冲区有数据
            data = ser.readline().decode().rstrip()  # 读取一行数据并移除换行符
            img=OG[290:790,560:1360]
            print("ABC")
            break
        # 按下ESC键退出循环
        if cv2.waitKey(1) == 27:
            break

# 释放VideoCapture对象和窗口
cap.release()
cv2.destroyAllWindows()

Sure
ABC


In [3]:
cv2.imshow("ABC",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
import cv2
import serial
import time 
# ser = serial.Serial('COM8', 9600)  # 根据实际情况修改串口号

# 创建VideoCapture对象，参数为摄像头索引号（通常为0，表示默认摄像头）
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)  # 设置帧宽度
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)  # 设置帧高度
ChackID=0
frames = []
Catch=0
start_time = None
while True:
    # 从摄像头捕获一帧图像
    if (ser.in_waiting and ChackID==0):  # 如果串口接收缓冲区有数据
        data = ser.readline().decode().rstrip()  # 读取一行数据并移除换行符
        print("Sure")  # 输出数据
        ChackID=1
    if(ChackID==1):
        ret, frame = cap.read()
        H=frame.shape[0]
        W=frame.shape[1]
        CH=H//2
        CW=W//2
        center=(CW,CH)
        OG=frame.copy()
        # frame=cv2.flip(frame, 1)
        # 如果成功捕获图像，则处理并显示
        cv2.rectangle(frame, (CW-350,CH-350), (CW+350,CH+350), (255, 0, 0), 5)
        # cv2.circle(frame,(50, 50), 15, (255, 0, 0), -1) # blue
        cv2.imshow('Camera', frame)
        if (ser.in_waiting and Catch==0):  # 如果串口接收缓冲区有数据
            data = ser.readline().decode().rstrip()  # 读取一行数据并移除换行符
            frames = []
            print("START")
            start_time =time.time()
            Catch=1
        if(Catch==1):
            frames.append(OG[CH-350:CH+350,CW-350:CW+350])
        
        # 按下ESC键退出循环
        if cv2.waitKey(1) == 27:
            break
        if start_time is not None and time.time() - start_time >= 5 and Catch==1:
            Catch=0
            print("STOP")

# 释放VideoCapture对象和窗口
cap.release()
cv2.destroyAllWindows()

Sure
START
STOP


In [ ]:
frames = np.stack(frames, axis=3)

In [21]:
for i, frame in enumerate(frames):
    cv2.imwrite(f"frame_{i}.jpg", frame)

In [23]:
temp=np.array(frames)
temp.shape


(151, 700, 700, 3)

In [33]:
import cv2
import socket
import pickle
import numpy as np

# 建立Socket連接
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
receiver_ip = '192.168.1.197'
receiver_port = 12345
s.connect((receiver_ip, receiver_port))


# 將影像序列化為字節流
frames = np.stack(frames, axis=3)
# img=cv2.imread("frame_101.jpg")
data = pickle.dumps(frames)

# 傳送影像資料
s.sendall(data)

# 關閉連接
s.close()

In [11]:
import cv2
import time 

# 设置摄像头索引，一般为0表示默认摄像头
camera_index = 0

# 创建VideoCapture对象，打开摄像头
cap = cv2.VideoCapture(camera_index)

# 创建空列表用于存储帧图像
frames = []

while True:
    # 读取摄像头的每一帧
    ret, frame = cap.read()

    # 检查是否成功读取帧
    if not ret:
        break

    # 将帧图像添加到列表中
    frames.append(frame)

    # 显示帧
    cv2.imshow("Frame", frame)

    # 按下 'q' 键退出循环
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放资源
cap.release()
cv2.destroyAllWindows()

# 遍历列表并保存每一帧图像



In [15]:
import numpy as np
temp=np.array(frames)

In [16]:
temp.shape

(191, 480, 640, 3)

In [4]:
import cv2

# 创建VideoCapture对象，参数为摄像头索引号（通常为0，表示默认摄像头）
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)  # 设置帧宽度
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)  # 设置帧高度
cx, cy = 200, 200  # 头部框中心坐标
rx, ry = 100, 150  # 长轴和短轴长度
while True:
    # 从摄像头捕获一帧图像
    ret, frame = cap.read()

    # 如果成功捕获图像，则处理并显示
    if ret:
        # 在图像上进行其他处理（例如，灰度化、边缘检测等）
        # ...

        # 显示图像
        cv2.ellipse(frame, (cx, cy), (rx, ry), 0, 0, 360, (0, 255, 0), 2)

        cv2.imshow('Camera', frame)

    # 按下ESC键退出循环
    if cv2.waitKey(1) == 27:
        break

# 释放VideoCapture对象和窗口
cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2
import serial
# ser = serial.Serial('COM8', 9600)  # 根据实际情况修改串口号

# 创建VideoCapture对象，参数为摄像头索引号（通常为0，表示默认摄像头）
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)  # 设置帧宽度
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)  # 设置帧高度
ChackID=1
while True:

    if(ChackID==1):
        ret, frame = cap.read()
        H=frame.shape[0]
        W=frame.shape[1]
        CH=H//2
        CW=W//2
        center=(CW,CH)
        OG=frame.copy()
        # frame=cv2.flip(frame, 1)
        # 如果成功捕获图像，则处理并显示
        # cv2.rectangle(frame, (CW-400,CH-250), (CW+400,CH+250), (255, 0, 0), 5)
        # cv2.circle(frame,(50, 50), 15, (255, 0, 0), -1) # blue
        cv2.imshow('Camera', frame)
        # 按下ESC键退出循环
        if cv2.waitKey(1) == 27:
            break

# 释放VideoCapture对象和窗口
cap.release()
cv2.destroyAllWindows()

In [ ]:
frame.shape

In [ ]:
cv2.imshow('Camera', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # 设置帧宽度
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)  # 设置帧高度
cv2.namedWindow('Camera', cv2.WINDOW_NORMAL)
while True:
    # 从摄像头捕获一帧图像
    ret, frame = cap.read()

    H,W=frame.shape[0:2]
    
    M=min(1920/W,1080/H)
    MW=int(M*W)
    MH=int(M*H)
    cv2.resizeWindow('Camera', int(M*W),int(M*H))
    # 如果成功捕获图像，则处理并显示
    if ret:
        # 在图像上进行其他处理（例如，灰度化、边缘检测等）
        # ...

        # 显示图像
        cv2.imshow('Camera', frame)

    # 按下ESC键退出循环
    if cv2.waitKey(1) == 27:
        break

# 释放VideoCapture对象和窗口
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2

# 创建VideoCapture对象，参数为摄像头索引号（通常为0，表示默认摄像头）
cap = cv2.VideoCapture(0)

# 定义放大倍数和放大镜半径
zoom_factor = 2  # 放大倍数
radius = 100     # 放大镜半径

while True:
    # 从摄像头捕获一帧图像
    ret, frame = cap.read()

    # 如果成功捕获图像，则处理并显示
    if ret:
        # 获取放大镜中心点坐标
        x, y = cv2.getWindowImageRect('Camera')[:2]
        center_x, center_y = x + frame.shape[1] // 2, y + frame.shape[0] // 2

        # 根据放大倍数和半径计算放大镜区域
        zoomed_area = frame[
            center_y - radius: center_y + radius,
            center_x - radius: center_x + radius
        ]

        # 调整放大镜区域大小为原始图像大小
        zoomed_area = cv2.resize(zoomed_area, (frame.shape[1], frame.shape[0]))

        # 在放大镜区域周围绘制圆形边框
        cv2.circle(frame, (center_x, center_y), radius, (0, 255, 0), 2)

        # 将放大镜区域与原始图像叠加显示
        result = cv2.addWeighted(frame, 0.7, zoomed_area, 0.3, 0)

        # 显示图像
        cv2.imshow('Camera', result)

    # 按下ESC键退出循环
    if cv2.waitKey(1) == 27:
        break

# 释放VideoCapture对象和窗口
cap.release()
cv2.destroyAllWindows()

In [ ]:
frame.shape

In [ ]:
cv2.namedWindow('My Image', cv2.WINDOW_NORMAL)
H,W=img.shape[0:2]
M=min(1920/W,1080/H)
cv2.resizeWindow('My Image', int(M*W),int(M*H))
cv2.imshow('My Image', img)
cv2.setMouseCallback('My Image', show_xy,img)  # 設定偵測事件的函式與視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
type(data)